![LogoUC3M](https://www.fundacion.uc3m.es/wp-content/uploads/2018/11/Logo-UC3M-nuevo.png)

### Aprendizaje Automático · Grado en Ingeniería Informática · Curso 2022/23
---
# **Cuaderno 1** - Práctica 1: Predicción de la producción de energía
***Grupo Reducido 82 - Grupo de laboratorio 13***






# **Carga de datos**


Para comenzar, importaremos las librerías que necesitamos:

+ `linear-tree`: Implementación de los modelos de árboles.
+ `statsmodels`: Para los intervalos de confianza.


In [21]:
!pip install --upgrade linear-tree
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Una vez instaladas las bibliotecas necesarias, importamos **Numpy** y **Pandas** ya que los necesitaremos para el desarrollo de la práctica.
Destacar que nos referiremos a **Numpy** como *np* y a **Pandas** como *pd* por simplicidad.

In [22]:
import numpy as np
import pandas as pd

A continuación, cargaremos los datos necesarios. Dado que somos el grupo 13, usaremos los dos conjuntos de datos correspondientes.</br></br>
Por una parte, tenemos los **datos disponibles**, que nos servirán para entrenar, 
evaluar y construir el modelo final y por otra parte los **datos de competición** sobre los que usaremos el modelo final para hacer las predicciones correspondientes.</br></br>

In [23]:
# Datos disponibles.
disp_df = pd.read_csv("disp_st13ns1.txt.bz2",
                      compression="bz2",
                      index_col=0)

# Datos competición.
comp_df = pd.read_csv("comp_st13ns1.txt.bz2",
                      compression="bz2",
                      index_col=0)

# Mostramos la información de cada conjunto de datos.
print(f"El conjunto de datos disponibles tiene {len(disp_df)} instancias.")
print(f"El conjunto de datos de competición tiene {len(disp_df)} instancias.")

El conjunto de datos disponibles tiene 4380 instancias.
El conjunto de datos de competición tiene 4380 instancias.


A continuación, separamos la **matriz de atributos** (**X**) y el **vector de la variable de respuesta** *salida* (**y**).

In [24]:
# Datos.
X = disp_df.drop('salida', axis=1)

# Etiquetas.
y = disp_df.salida

# **Análisis Exploratorio de Datos (EDA)**
Antes de comenzar, destacar que dado que los resultados deben ser reproducibles, hemos fijado la **semilla de números aleatorios** en los lugares adecuados. Para ello, hemos seleccionado como semilla el número del grupo de prácticas (*13*).</br></br>
Para poder realizar el análisis exploratorio de datos, llevaremos a cabo los siguientes pasos:

+ Número de instancias y atributos.
+ Tipo de atributos (numéricos o categóricos).
+ Missing values.
+ Tipo de problema (clasificación o regresión).



EDA steps (from T1_SKlearnTrees.ipynb):
+ How many instances and attributes there are
+ What type of attributes there are (numerical or categorical). This is done to check whether there are categorical features that should be encoded (as dummies / one-hot-encoding)
+ What attributes have missing values, and how many
+ Whether it is a classification or a regression problem (response variable), and in case of classification, whether the class is imbalanced.

### **Número de instancias y atributos**
Los **datos disponibles** cuentan con 75 atributos y la variable de respuesta *salida*. Contiene 12 años de datos considerando una instancia por día y todos los años de 365 días. Por tanto, contamos con un total de **4380 instancias**.</br></br>

Los **datos de competición** cuentan con las mismas 75 variables de entrada que los datos disponibles y se trata de un conjunto con 2 años de datos y una instancia por día. En este caso, no se proporciona la variable de respuesta salida, dado que usaremos nuestro modelo final para hacer predicciones sobre estos datos como ya hemos indicado antes. Por tanto, contamos con un total de **733 instancias**.</br></br>

A continuación, profundizaremos más en los atributos, su significado y nomenclatura.</br></br>

Como podemos ver en la siguiente tabla, contamos con **15 atributos** diferentes, cada uno de ellos con su respectivo nombre (identificador) y con su unidad concreta. Estos atributos corresponden con **variables meteorológicas** diversas.</br></br>

| Atributo | Descripción | Unidades |
| :- | :- | -: |
| `apcp_sfc` | Precipitación acumulada de 3 horas en la superficie. | $\dfrac{kg}{m^2}$ |
| `dlwrf_sfc` | Promedio de flujo radiativo de onda larga hacia abajo en la superficie. | $\dfrac{W}{m^2}$ |
| `dswrf_sfc` | Promedio de flujo radiativo de onda corta hacia abajo en la superficie. | $\dfrac{W}{m^2}$ |
| `pres_msl` | La presión del aire al nivel medio del mar. | $Pa$ |
| pwat_eatm	 | Agua precipitable sobre toda la profundidad de la atmósfera. | $\dfrac{kg}{m^2}$ |
| `spfh_2m`	 | Humedad específica a 2 m sobre el suelo. | $\dfrac{kg}{kg}$ |
| `tcdc_eatm`	 | Cobertura total de nubes sobre toda la profundidad de la atmósfera. | $\%$ |
| `tcolc_eatm` | Condensado total integrado en la columna sobre toda la atmósfera. | $\dfrac{kg}{m^2}$ |
| `tmax_2m` | Temperatura máxima en las últimas 3 horas a 2 m sobre el suelo. | $K$ |
| `tmin_2m` | Temperatura mínima en las últimas 3 horas a 2 m sobre el suelo. | $K$ |
| `tmp_2m` | Temperatura actual a 2 m sobre el suelo.	| $K$ |
| `tmp_sfc` | Temperatura de la superficie. | $K$ |
| `ulwrf_sfc` | Radiación ascendente de onda larga en la superficie. | $\dfrac{kg}{m^2}$ |
| `ulwrf_tatm` | Radiación ascendente de onda larga en la parte superior de la atmósfera. | $\dfrac{kg}{m^2}$ |
| `uswrf_sfc` | Radiación ascendente de onda corta en la superficie. | $\dfrac{kg}{m^2}$ |

</br>Partiendo de los 15 atributos que podemos observar en la tabla anterior, se indica una variable en los datos para cada uno ellos en 5 momentos del día. Esto es, contamos con:</br></br>

$5\;instantes\;de\;tiempo\;\cdot\;15\;variables\;meteorológicas\;=\;75\;variables\;de\;entrada$

</br>Los instantes de tiempo vienen dados por una numeración del 1 al 5:
+ 1 $\rightarrow$ 12h
+ 2 $\rightarrow$ 15h
+ 3 $\rightarrow$ 18h
+ 4 $\rightarrow$ 21h
+ 5 $\rightarrow$ 24h

</br>Por tanto, cada variable meteorológica aparece en el conjunto de datos 5 veces, una por cada instante de tiempo. El instante de tiempo se indica en el subíndice $i$, donde $i=1,2,3,4,5$. Por ejemplo, $apcp\_sfc_1$ es la variable que indica la precipitación acumulada de 3 horas en la superficie a las 12h (instante de tiempo 1).

## **Tipo de atributos**.
Dentro de los diferentes tipos de atributos de los datos podemos encontrar los categóricos, numéricos u ordinales.</br></br>

En el caso de nuestros datos, podemos ver como claramente todos ellos son **atributos numéricos reales** a excepción de la variable de respuesta *salida*, la cual es un **atributo numérico entero**.</br></br>

Por tanto, dado que todos los datos con los que vamos a tratar son atributos numéricos, no tendremos que transformar ninguna variable categórica en one-hot-encoding, dado que no vamos a lidiar con ninguna variable de este tipo.


In [25]:
# Mostramos todos los datos.
print('Matriz de atributos:\n\n', X)
print('\n\nVector de la variable de respuesta:\n\n', y)

# Mostramos el tipo de dato de una variable meteorológica y de un valor de la variable de respuesta.
print('\nEjemplo de tipo de dato de variable meteorológica:', type(X['apcp_sf1_1'][0]))
print('Ejemplo de tipo de dato de variable de respuesta:', type(y[0]))

Matriz de atributos:

        apcp_sf1_1  apcp_sf2_1  apcp_sf3_1  apcp_sf4_1  apcp_sf5_1  dlwrf_s1_1  \
V1       0.000000    0.000000    0.000000    0.000000    0.000000  276.583582   
V2       0.000000    0.000000    0.011818    0.037273    0.543636  249.089505   
V3       0.014545    0.000000    0.000000    0.000000    0.002727  229.461820   
V4       0.000000    0.009091    0.000000    0.000000    0.000000  239.590321   
V5       0.000000    0.000000    0.000000    0.000000    0.000000  230.253657   
...           ...         ...         ...         ...         ...         ...   
V4376    0.000909    0.114545    0.352727    0.882727    1.379091  284.532288   
V4377    0.000909    0.000000    0.000000    0.000000    0.000000  251.460183   
V4378    0.000000    0.000000    0.000000    0.000000    0.000000  268.446164   
V4379    0.000000    0.000000    0.000000    0.000000    0.000000  268.225686   
V4380    0.000000    0.000000    0.000000    0.000000    0.000000  268.385348   

    

Como se puede comprobar en las dos últimas llamadas realizadas, las variables de respuesta utilizan atributos del tipo `numpy.float64`, siendo estos valores **numéricos reales**. Por otra parte, la variable de respuesta *salida* utiliza valores del tipo `numpy.int64`, siendo estos valores **numéricos enteros**.


### **Missing values**.
(Investigar esto)

### **Tipo de problema**.
Como hemos podido ver anteriormente mostrando los datos en pantalla, este problema es claramente de **regresión**, dado que tendremos que construir un modelo para realizar **predicción numérica**.

# **División de datos**
Como se puede ver en el código a continuación, dividiremos los datos de acuerdo con el enunciado establecido. Esto es, utilizaremos los **10 primeros años** del conjunto de datos disponibles para **entrenamiento** (*train*) y los **2 últimos años** para **pruebas** (*test*).</br></br>

Dado que se nos pide que no desordenemos los datos antes de partir en entrenamiento y test para poder respestar el orden temporal, usaremos el parámetro `shuffle = False` al invocar a la función `train_test_split` de Scikit-learn.

In [26]:
from sklearn.model_selection import train_test_split

# Entrenamiento (10 primeros años) y test (2 últimos años).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2/12, random_state=13, shuffle=False)

# !!! REVISAR !!!
# ¿V1 ES EL DATO MÁS ANTIGUO O EL MÁS NUEVO?
# !!! REVISAR !!!

# Comprobamos que los datos se hayan dividido como queremos.
print(X_train.shape, y_train.shape)   # 3650 días -> 10 años.
print(X_test.shape, y_test.shape)     # 720 días  ->  2 años.


(3650, 75) (3650,)
(730, 75) (730,)


Como se puede ver y, como hemos explicado anteriormente, dado que los resultados deben ser reproducibles, hemos fijado la **semilla de números aleatorios** en los lugares adecuados. Para ello, hemos seleccionado como semilla el número del grupo de prácticas (*13*) mediante `random_state = 13`.

# **Métodos Básicos**
Para los modelos básicos, consideraremos los siguientes **métodos básicos**:
+ KNN
+ Árboles de regresión
+ Regresión lineal

Además, destacar que las métricas de evaluación son **RMSE** y **MAE**.

## **Evaluación con hiperparámetros por omisión**

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.linear_model import LinearRegression

from sklearn import metrics

### **KNN**

In [28]:
clf = KNeighborsClassifier()
np.random.seed(13)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc_knn = metrics.accuracy_score(y_test, y_pred)
print(f"Precisión del modelo KNN: {acc_knn * 100:.2f} %")
np.random.seed(13)

Precisión del modelo KNN: 0.00 %


### **Árboles de regresión**

In [29]:
clf = tree.DecisionTreeClassifier()
np.random.seed(13)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [30]:
# Predicciones del conjunto de test.
y_pred = clf.predict(X_test)
print(y_pred)

[12637200  9778800  1818900  8421300 12719700  6144000  9705000 21749100
 21146400 10009500 11848500 11940600 12079200  2212800  3978900  4897200
  7211700  1487100  1662300  6144000 11891400 11130300  8176800 11826900
 13606500  6573900  6539400  7719300 11661300 14392200  7719300 14388600
  7194600  3636900  8469300 11654400  1831500  2072700  8710800 20500200
 15897600  2935800  5950800  4215300 14152800  9077400  2721900 21933600
  7604700 15738000 24406200  9674400  9658200 16991100 12695400  8167200
 13623000 11053500 17104500 26765100 13348800   968700 18207900  5726100
  1806000  1544400 26625300 23575800 20580300 16065900 14466300 20161800
 23229600 15870600  9819600 21666300  8058000 22200300 18462900 17576100
  5618100  3084300 20804700 25643100 17886900  4380300  3399900  6357900
 11725500 22123800 14523000 19789800 16587600 17713500 17886900 20454000
 23896500 14696100 14368800  4029300 20994000 18600900  3588000 13656900
 11241900  6119100 18740100 21036600  8911800 23275

In [31]:
# Comparación de predicciones actuales con 
print(np.hstack((y_pred[:5,np.newaxis], y_test[:5,np.newaxis])))

[[12637200 11121600]
 [ 9778800 10962000]
 [ 1818900  1597200]
 [ 8421300 11056200]
 [12719700  8856600]]


<ipython-input-31-2f31e14bb28e>:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  print(np.hstack((y_pred[:5,np.newaxis], y_test[:5,np.newaxis])))


In [34]:
accuracy_tree = metrics.accuracy_score(y_test, y_pred)
print(print(f"Precisión del modelo árboles de regresión: {accuracy_tree * 100:.2f} %"))

Precisión del modelo árboles de regresión: 0.00 %
None


### **Regresión lineal**

## **Ajuste de hiperparámetros**

### **KNN**

### **Árboles de regresión**

### **Regresión lineal**

## **Conclusiones**
Responder a:
+ ¿Cuál es el mejor método?
+ ¿Cuál de los métodos
básicos de aprendizaje automático es más rápido?
+ ¿Los resultados son mejores que los
regresores triviales/naive/baseline?
+ ¿El ajuste de hiperparámetros mejora con respecto a los
valores por omisión?
+ ¿Hay algún equilibrio entre tiempo de ejecución y mejora de
resultados? 

## **Reducción de dimensionalidad**

# **Métodos avanzados**

## **Evaluación con hiperparámetros por omisión**

### **SVMs**

### **Random Forests**

## **Ajuste de hiperparámetros**

### **SVMs**

### **Random Forests**

## **Interpretación de la importancia de los atributos**

## **Conclusiones**

Como podemos ver en el resultado anterior, la precisión (accuracy) del árbol creado es 0.0, por lo que es nula. Por tanto, se puede ver como el primer modelo claramente no es adecuado para nuestros datos. De esta forma, tendremos que estudiar otro tipo de modelo.